This notebook contains code from the other notebooks that I didn't want to keep in them, but also didn't want to lose.

$\frac{\rho}{\rho_b}=\frac{P}{P_b}\cdot\frac{T_b}{T}$

In [ ]:
def saturation_pressure_of_water_vapor(temperature_C):
    """
    Estimates the saturation pressure (kPa) of water vapor for a given temperature (C)
    using Teten's fomula, described: https://en.wikipedia.org/wiki/Tetens_equation
    """
    if(temperature_C > 0):
        return 0.61078 * np.exp((17.27 * temperature_C) / (temperature_C + 237.3))
    else:
        return 0.61078 * np.exp((21.875 * temperature_C) / (temperature_C + 265.5))
    
def water_vapor_pressure(temperature_C, relative_humidity):
    """
    Relative humidity is defined as the ratio (expressed as a percentage) of the actual vapor pressure to
    the saturation vapor pressure at a given temperature.
    """
    return saturation_pressure_of_water_vapor(temperature_C) * (relative_humidity/100) * 1000
    
print(f"Es(30) = {saturation_pressure_of_water_vapor(30)}")
print(f"Es(-35) = {saturation_pressure_of_water_vapor(-30)}")

water_vapor_pressure(-56, 40)

In [ ]:
gas_constant_dry_air = 287.053 # J/kg*K

# From: https://commons.erau.edu/cgi/viewcontent.cgi?article=1124&context=ijaaa
def calc_altitude(pressure_Pa, temperature_C, humidity_RH):
    temperature_K = temperature_C + 273.15
    layer = isa['base_pressure_Pa'].searchsorted(pressure_Pa)
    parameters = isa.iloc[layer]
    
    base_altitude_m = parameters['geopotential_altitude_m']
    base_temperature_K = parameters['base_temperature_K']
    temp_ratio = base_temperature_K / temperature_K
    
    vapor_pressure = water_vapor_pressure(temperature_C, humidity_RH)
    moist_air_pressure = pressure_Pa + vapor_pressure
    pressure_ratio = moist_air_pressure / parameters['base_pressure_Pa']
    
    density_ratio = pressure_ratio * temp_ratio
#    print(density_ratio)
    lapse_rate = parameters['lapse_rate_K_m']
    
    #TODO: What does this represent, whats a good name?
    RdL = dry_air_gas_constant * lapse_rate
    foo = -1.0 * (RdL) / (gravitational_acceleration + RdL)
#    print(foo)
    
    if lapse_rate == 0:
        factor = altitude_constants * base_temperature_K
#        print(factor)
        return base_altitude_m + factor * math.log(density_ratio)
    else:
        factor = base_temperature_K / lapse_rate
#        print(factor)
        exponent = altitude_constants * lapse_rate
#        print(exponent)
        return base_altitude_m + factor * (pow(density_ratio, exponent)-1)
    
calc_altitude = np.vectorize(calc_altitude)

print(calc_altitude(108900, 19, 20))
print(calc_altitude(108900, 19, 30))
print(calc_altitude(108900, 19, 40))

print()
print(calc_altitude(50000, 19, 0))
print()
print(calc_altitude(22633, 19, 20))